In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ec089df4cb4347639a1be1083c855aa8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_H7kb_ZHQnxS7PiUszQCPTapdl4iYUSw5C1MVKRw9_Rm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_ec089df4cb4347639a1be1083c855aa8.get_object(Bucket='reciperecognitionusindibmwatson-donotdelete-pr-uudijq95enrzb1', Key='dataset-20220814T164619Z-001.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:

ls-1

dataset/


In [6]:
%cd /home/wsuser/work/

/home/wsuser/work


In [7]:
ls-1

dataset/


In [8]:
pwd

'/home/wsuser/work'

In [9]:
x = train_datagen.flow_from_directory(r"/home/wsuser/work/dataset/training_set",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2418 images belonging to 3 classes.


In [10]:
y= test_datagen.flow_from_directory(r"/home/wsuser/work/dataset/test_set",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 600 images belonging to 3 classes.


In [11]:
model=Sequential()

In [12]:
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3)))

In [13]:
model.add(MaxPool2D(2,2))

In [14]:
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))  

In [15]:
model.add(Dropout(0.2))

In [16]:
model.add(Flatten())

In [17]:
model.add(Dense(32))

In [18]:
model.add(Dense(3,activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
model.fit_generator(
  x,
  validation_data=y,
  epochs=5,
  steps_per_epoch=len(x),
  validation_steps=len(y)
)

/tmp/wsuser/ipykernel_164/2473025506.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
76/76 [==============================] - 20s 256ms/step - loss: 1.0493 - accuracy: 0.4256 - val_loss: 0.9175 - val_accuracy: 0.5283
Epoch 2/5
76/76 [==============================] - 19s 252ms/step - loss: 0.8966 - accuracy: 0.5678 - val_loss: 0.9440 - val_accuracy: 0.5400
Epoch 3/5
76/76 [==============================] - 19s 250ms/step - loss: 0.8481 - accuracy: 0.6042 - val_loss: 0.8184 - val_accuracy: 0.6100
Epoch 4/5
76/76 [==============================] - 19s 249ms/step - loss: 0.7494 - accuracy: 0.6787 - val_loss: 0.8465 - val_accuracy: 0.6133
Epoch 5/5
76/76 [==============================] - 19s 249ms/step - loss: 0.7085 - accuracy: 0.6960 - val_loss: 0.7218 - val_accuracy: 0.6800


In [21]:
model.save("food.h5")